### Importação das bibliotecas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install plotly --upgrade

In [ ]:
!pip install optbinning

In [ ]:
import pandas as pd
import gc
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from optbinning import BinningProcess
from optbinning import OptimalBinning

In [ ]:
warnings.filterwarnings("ignore", category=FutureWarning)

### Dados

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# Dados
microdados = pd.read_parquet('/content/drive/MyDrive/PIBIC/dadosCenso2018_ComNovasColunas_WoE.parquet')

In [ ]:
microdados.head()

#### Modelagem e Transformando em Binário

In [ ]:
# Concatenando as datas de nascimento, para formar uma coluna para data
microdados['DATA_NASCIMENTO'] = (
    microdados['NU_DIA_NASCIMENTO'].astype(str).str.zfill(2) + '/' +
    microdados['NU_MES_NASCIMENTO'].astype(str).str.zfill(2) + '/' +
    microdados['NU_ANO_NASCIMENTO'].astype(str)
)

# Exibir os primeiros registros
microdados[["NU_DIA_NASCIMENTO", "NU_MES_NASCIMENTO", "NU_ANO_NASCIMENTO", "DATA_NASCIMENTO"]].head()

In [ ]:
# Os valores Nan são valores vazios, e senão estão preenchidos então podemos atribuir a pessoas que não tem financiamento "0"
# Afim, de transformar o atributo sensível para binário
microdados['IN_FINANCIAMENTO_ESTUDANTIL_BINARIO'] = microdados['IN_FINANCIAMENTO_ESTUDANTIL'].fillna(0)

In [ ]:
microdados['IN_FINANCIAMENTO_ESTUDANTIL_BINARIO'].unique()

In [ ]:
microdados['IN_DEFICIENCIA'].unique()

In [ ]:
# 0 -> Não tem deficiência
# 1 -> Tem deficiência
# 9 -> Não possui a informação

In [ ]:
microdados['IN_DEFICIENCIA_BINARIO'] = microdados['IN_DEFICIENCIA'].replace(9, 0)

In [ ]:
microdados.drop(['IN_FINANCIAMENTO_ESTUDANTIL', 'IN_DEFICIENCIA'], axis=1, inplace=True)

In [ ]:
# Salvando os microdados no Drive
microdados.to_parquet('/content/drive/MyDrive/PIBIC/dadosCenso2018_ComNovasColunas_WoE.parquet')

#### Concatenações

Devido à alta demanda computacional, realizar as concatenações diretamente no código não era eficiente e sobrecarregava a memória RAM do Google Colab. Uma estratégia para reduzir essa sobrecarga foi executar algumas das concatenações antecipadamente, em vez de realizá-las todas em conjunto durante a execução do código.

##### CO_CINE_AREA GERAL

In [ ]:
microdados['CINE_MUNICIPIO'] = (
    microdados['CO_CINE_AREA_GERAL'].astype(str) + "_" + microdados['CO_MUNICIPIO_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['CINE_ANO'] = (
    microdados['CO_CINE_AREA_GERAL'].astype(str) + "_" + microdados['NU_ANO_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['CINE_DIA'] = (
    microdados['CO_CINE_AREA_GERAL'].astype(str) + "_" + microdados['NU_ANO_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['CINE_ANO_DIA'] = (
    microdados['CINE_ANO'].astype(str) + "_" + microdados['NU_DIA_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['CINE_DATA'] = (
    microdados['CO_CINE_AREA_GERAL'].astype(str) + "_" + microdados['DATA_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['CINE_DATA_MUNICIPIO'] = (
    microdados['CINE_DATA'].astype(str) + "_" + microdados['CO_MUNICIPIO_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['CINE_DATA_MUNICIPIO_RACA'] = (
    microdados['CINE_DATA_MUNICIPIO'].astype(str) + "_" + microdados['TP_COR_RACA'].astype(str)
)

##### CO_MUNICIPIO

In [ ]:
microdados['CO_MUNICIPIO_MUNICIPIO'] = (
    microdados['CO_MUNICIPIO'].astype(str) + "_" + microdados['CO_MUNICIPIO_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['CO_MUNICIPIO_ANO'] = (
    microdados['CO_MUNICIPIO'].astype(str) + "_" + microdados['NU_ANO_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['CO_MUNICIPIO_DIA'] = (
    microdados['CO_MUNICIPIO'].astype(str) + "_" + microdados['NU_DIA_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['CO_MUNICIPIO_ANO_DIA'] = (
    microdados['CO_MUNICIPIO_ANO'].astype(str) + "_" + microdados['NU_DIA_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['CO_MUNICIPIO_DATA'] = (
    microdados['CO_MUNICIPIO'].astype(str) + "_" + microdados['DATA_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['CO_MUNICIPIO_DATA_MUNICIPIO'] = (
    microdados['CO_MUNICIPIO_DATA'].astype(str) + "_" + microdados['CO_MUNICIPIO_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['CO_MUNICIPIO_DATA_MUNICIPIO_RACA'] = (
    microdados['CO_MUNICIPIO_DATA_MUNICIPIO'].astype(str) + "_" + microdados['TP_COR_RACA'].astype(str)
)

##### CO_UF

In [ ]:
microdados['UF_MUNICIPIO'] = (
    microdados['CO_UF'].astype(str) + "_" + microdados['CO_MUNICIPIO_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['UF_ANO'] = (
    microdados['CO_UF'].astype(str) + "_" + microdados['NU_ANO_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['UF_DIA'] = (
    microdados['CO_UF'].astype(str) + "_" + microdados['NU_DIA_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['UF_ANO_DIA'] = (
    microdados['UF_ANO'].astype(str) + "_" + microdados['NU_DIA_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['UF_DATA'] = (
    microdados['CO_UF'].astype(str) + "_" + microdados['DATA_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['UF_DATA_MUNICIPIO'] = (
    microdados['UF_DATA'].astype(str) + "_" + microdados['CO_MUNICIPIO_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['UF_DATA_MUNICIPIO_RACA'] = (
    microdados['UF_DATA_MUNICIPIO'].astype(str) + "_" + microdados['TP_COR_RACA'].astype(str)
)

##### CO_IES

In [ ]:
microdados['IES_MUNICIPIO'] = (
    microdados['CO_IES'].astype(str) + "_" + microdados['CO_MUNICIPIO_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['IES_ANO'] = (
    microdados['CO_IES'].astype(str) + "_" + microdados['NU_ANO_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['IES_DIA'] = (
    microdados['CO_IES'].astype(str) + "_" + microdados['NU_DIA_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['IES_ANO_DIA'] = (
    microdados['IES_ANO'].astype(str) + "_" + microdados['NU_DIA_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['IES_DATA'] = (
    microdados['CO_IES'].astype(str) + "_" + microdados['DATA_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['IES_DATA_MUNICIPIO'] = (
    microdados['IES_DATA'].astype(str) + "_" + microdados['CO_MUNICIPIO_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['IES_DATA_MUNICIPIO_RACA'] = (
    microdados['IES_DATA_MUNICIPIO'].astype(str) + "_" + microdados['TP_COR_RACA'].astype(str)
)

##### TP_CATEGORIA_ADMINISTRATIVA

In [ ]:
microdados['CATEGORIA_MUNICIPIO'] = (
    microdados['TP_CATEGORIA_ADMINISTRATIVA'].astype(str) + "_" + microdados['CO_MUNICIPIO_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['CATEGORIA_ANO'] = (
    microdados['TP_CATEGORIA_ADMINISTRATIVA'].astype(str) + "_" + microdados['NU_ANO_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['CATEGORIA_DIA'] = (
    microdados['TP_CATEGORIA_ADMINISTRATIVA'].astype(str) + "_" + microdados['NU_DIA_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['CATEGORIA_ANO_DIA'] = (
    microdados['CATEGORIA_ANO'].astype(str) + "_" + microdados['NU_DIA_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['CATEGORIA_DATA'] = (
    microdados['TP_CATEGORIA_ADMINISTRATIVA'].astype(str) + "_" + microdados['DATA_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['CATEGORIA_DATA_MUNICIPIO'] = (
    microdados['CATEGORIA_DATA'].astype(str) + "_" + microdados['CO_MUNICIPIO_NASCIMENTO'].astype(str)
)

In [ ]:
microdados['CATEGORIA_DATA_MUNICIPIO_RACA'] = (
    microdados['CATEGORIA_DATA_MUNICIPIO'].astype(str) + "_" + microdados['TP_COR_RACA'].astype(str)
)

### Experimento 5 - WoE e IV

In [ ]:
def woe_iv_simples(dados, var_categorica, target_binario):
    """
    Calcula WoE e IV para uma variável categórica usando OptimalBinning.

    Parâmetros:
        dados (DataFrame): Base de dados contendo as variáveis.
        var_categorica (str): Nome da variável categórica a ser analisada.
        target_binario (str): Nome da variável alvo binária (0/1).

    Retorna:
        df_table (DataFrame): Tabela com bins, WoE, IV, event rate, etc.
        total_iv (float): Valor total do IV da variável.
    """

    X = dados[[var_categorica]].astype(str)
    y = dados[target_binario]

    optb = OptimalBinning(name=var_categorica, dtype="categorical")
    optb.fit(X[var_categorica].values, y.values)

    table = optb.binning_table.build()

    df_table = pd.DataFrame({
        "Bin": table["Bin"],
        "Count": table["Count"],
        "Event Rate": table["Event rate"],
        "WoE": table["WoE"],
        "IV": table["IV"],
        "Count (%)": table["Count (%)"]
    }).round({"Event Rate": 3, "WoE": 3, "IV": 4, "Count (%)": 2})

    # Filtrar bins válidos (exclui Missing, Special e Total)
    df_bins = df_table[
        ~df_table["Bin"].isin(["Missing", "Special"]) &
        (df_table["Count (%)"] < 1.0)  # Evita a linha de total (100%)
    ]

    # Calcular IV apenas dos bins válidos
    total_iv = round(df_bins["IV"].sum(), 4)

    print(f"📊 Variable: {var_categorica}\n")
    print(df_table.to_string(index=False), "\n")
    print(f"🔎 Total IV: {total_iv}")

    return

In [ ]:
def woe_iv_combinacoes_grandes(dados, variaveis, target_binario, max_unique=300, show_rows=10, sample_frac=0.5):

    # Reduz volume com amostragem estratificada
    dados_sample, _ = train_test_split(dados, train_size=sample_frac, stratify=dados[target_binario], random_state=42)

    X = dados_sample[variaveis].astype(str)
    y = dados_sample[target_binario]

    nome_nova_var = "_".join(variaveis)
    X[nome_nova_var] = X[variaveis[0]]

    for var in variaveis[1:]:
      X[nome_nova_var] = np.char.add(np.char.add(X[nome_nova_var].values, "_"), X[var].values)

    binning_process = BinningProcess(variable_names=[nome_nova_var])
    binning_process.fit(X[[nome_nova_var]], y)

    binned_var = binning_process.get_binned_variable(nome_nova_var)
    table = binned_var.binning_table.build()

    df_table = pd.DataFrame({
        "Bin": table["Bin"],
        "Count": table["Count"],
        "Event Rate": table["Event rate"],
        "WoE": table["WoE"],
        "IV": table["IV"],
        "Count (%)": table["Count (%)"]
    }).round({"Event Rate": 3, "WoE": 3, "IV": 4, "Count (%)": 2})

    # Filtrar bins válidos (exclui Missing, Special e Total)
    df_bins = df_table[
        ~df_table["Bin"].isin(["Missing", "Special"]) &
        (df_table["Count (%)"] < 1.0)  # Evita a linha de total (100%)
    ]

    # Calcular IV apenas dos bins válidos
    total_iv = round(df_bins["IV"].sum(), 4)

    print(f"📊 Variable: {' + '.join(variaveis)}")
    print(df_table.head(show_rows).to_string(index=False), "\n")
    print(f"🔎 Total IV: {total_iv}\n")

    return

#### Atributo Sensível [IN_FINANCIAMENTO_ESTUDANTIL_BINARIO]

##### 1º Abordagem

###### CO_CURSO

In [ ]:
woe_iv_simples(microdados, 'CO_CURSO', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CURSO_MUNICIPIO', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CURSO_ANO', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CURSO_ANO_DIA', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CURSO_DATA', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CURSO_DATA_MUNICIPIO', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CURSO_DATA_MUNICIPIO_RACA', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_IES', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

##### 2º Abordagem

###### CO_CINE_AREA_GERAL

In [ ]:
woe_iv_simples(microdados, 'CO_CINE_AREA_GERAL', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CINE_MUNICIPIO', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CINE_ANO_DIA', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CINE_DATA', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CINE_DATA_MUNICIPIO', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CINE_DATA_MUNICIPIO_RACA', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CINE_AREA_GERAL'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CINE_AREA_GERAL', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_CINE_AREA_GERAL', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_CINE_AREA_GERAL', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_IES', 'CO_CINE_AREA_GERAL', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'CO_CINE_AREA_GERAL', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

###### CO_MUNICIPIO

In [ ]:
woe_iv_simples(microdados, 'CO_MUNICIPIO', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CO_MUNICIPIO_MUNICIPIO', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CO_MUNICIPIO_ANO_DIA', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CO_MUNICIPIO_DATA', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CO_MUNICIPIO_DATA_MUNICIPIO', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CO_MUNICIPIO_DATA_MUNICIPIO_RACA', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_MUNICIPIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_MUNICIPIO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_MUNICIPIO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_MUNICIPIO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_IES', 'CO_MUNICIPIO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'CO_MUNICIPIO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

###### CO_UF

In [ ]:
woe_iv_simples(microdados, 'CO_UF', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'UF_MUNICIPIO', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'UF_ANO_DIA', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'UF_DATA', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'UF_DATA_MUNICIPIO', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'UF_DATA_MUNICIPIO_RACA', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_UF'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_UF', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_UF', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_UF', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_IES', 'CO_UF', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'CO_UF', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

###### CO_IES

In [ ]:
woe_iv_simples(microdados, 'CO_IES', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'IES_MUNICIPIO', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'IES_ANO_DIA', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'IES_DATA', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'IES_DATA_MUNICIPIO', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'IES_DATA_MUNICIPIO_RACA', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_IES'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_IES', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_IES', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_IES', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

###### TP_CATEGORIA_ADMINISTRATIVA

In [ ]:
woe_iv_simples(microdados, 'TP_CATEGORIA_ADMINISTRATIVA', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CATEGORIA_MUNICIPIO', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CATEGORIA_ANO_DIA', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CATEGORIA_DATA', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CATEGORIA_DATA_MUNICIPIO', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CATEGORIA_DATA_MUNICIPIO_RACA', 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_CATEGORIA_ADMINISTRATIVA'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_CATEGORIA_ADMINISTRATIVA', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'TP_CATEGORIA_ADMINISTRATIVA', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'TP_CATEGORIA_ADMINISTRATIVA', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_IES', 'TP_CATEGORIA_ADMINISTRATIVA', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'TP_CATEGORIA_ADMINISTRATIVA', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_FINANCIAMENTO_ESTUDANTIL_BINARIO')

#### Atributo Sensível [IN_DEFICIENCIA_BINARIO]

##### 1º Abordagem

###### CO_CURSO

In [ ]:
woe_iv_simples(microdados, 'CO_CURSO', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CURSO_ANO', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CURSO_ANO_DIA', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CURSO_DATA', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CURSO_DATA_MUNICIPIO', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CURSO_DATA_MUNICIPIO_RACA', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_IES', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

##### 2º Abordagem

###### CO_CINE_AREA_GERAL

In [ ]:
woe_iv_simples(microdados, 'CO_CINE_AREA_GERAL', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CINE_MUNICIPIO', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CINE_DIA', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CINE_ANO_DIA', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CINE_DATA', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CINE_DATA_MUNICIPIO', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CINE_DATA_MUNICIPIO_RACA', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CINE_AREA_GERAL'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CINE_AREA_GERAL', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_CINE_AREA_GERAL', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_CINE_AREA_GERAL', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_IES', 'CO_CINE_AREA_GERAL', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'CO_CINE_AREA_GERAL', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

###### CO_MUNICIPIO

In [ ]:
woe_iv_simples(microdados, 'CO_MUNICIPIO', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CO_MUNICIPIO_MUNICIPIO', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CO_MUNICIPIO_DIA', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CO_MUNICIPIO_ANO_DIA', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CO_MUNICIPIO_DATA', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CO_MUNICIPIO_DATA_MUNICIPIO', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CO_MUNICIPIO_DATA_MUNICIPIO_RACA', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_MUNICIPIO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_MUNICIPIO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_MUNICIPIO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_MUNICIPIO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_IES', 'CO_MUNICIPIO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'CO_MUNICIPIO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

###### CO_UF

In [ ]:
woe_iv_simples(microdados, 'CO_UF', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'UF_MUNICIPIO', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'UF_DIA', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'UF_ANO_DIA', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'UF_DATA', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'UF_DATA_MUNICIPIO', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'UF_DATA_MUNICIPIO_RACA', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_UF'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_UF', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_UF', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_UF', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_IES', 'CO_UF', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'CO_UF', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

###### CO_IES

In [ ]:
woe_iv_simples(microdados, 'CO_IES', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'IES_MUNICIPIO', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'IES_DIA', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'IES_ANO_DIA', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'IES_DATA', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'IES_DATA_MUNICIPIO', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'IES_DATA_MUNICIPIO_RACA', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_IES'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_IES', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_IES', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_IES', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

###### TP_CATEGORIA_ADMINISTRATIVA

In [ ]:
woe_iv_simples(microdados, 'TP_CATEGORIA_ADMINISTRATIVA', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CATEGORIA_MUNICIPIO', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CATEGORIA_DIA', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CATEGORIA_ANO_DIA', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CATEGORIA_DATA', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CATEGORIA_DATA_MUNICIPIO', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
woe_iv_simples(microdados, 'CATEGORIA_DATA_MUNICIPIO_RACA', 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_CATEGORIA_ADMINISTRATIVA'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_CATEGORIA_ADMINISTRATIVA', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'TP_CATEGORIA_ADMINISTRATIVA', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'TP_CATEGORIA_ADMINISTRATIVA', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_IES', 'TP_CATEGORIA_ADMINISTRATIVA', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')

In [ ]:
# amostragem - 50% dos dados
woe_iv_combinacoes_grandes(microdados, ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'TP_CATEGORIA_ADMINISTRATIVA', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO'], 'IN_DEFICIENCIA_BINARIO')